In [1]:
from data_loading.data_loader import SciBot_DataLoader 
from data_loading.mapping_loader import ScibotMappingLoader
from os.path import join as pjoin

In [2]:
from config import ARTICLES_DIR, GAZEDATA_VERSION, GAZE_DIR, MAPPING_DIR


In [47]:
article_ids = list(dataloader.grel_articles.keys())
article_id = article_ids[3]

In [48]:
dataloader = SciBot_DataLoader(pjoin(GAZE_DIR, GAZEDATA_VERSION),  gaze_data=True, googleNQ=True, gREL=True, 
                                include_users=['A04'], article_dir=ARTICLES_DIR)

mappingloader = ScibotMappingLoader(MAPPING_DIR, "g-REL", article_id)

In [49]:
import numpy as np 

for aid in article_ids:
    print(aid, " ", np.unique(dataloader.grel_reading['A04'][aid[:-2]]['dataframe']['paragraph_id']))


g-rel_q075-1_i   [-2  0]
g-rel_q076-1_r   [-2  0]
g-rel_q085-2_i   [-2  0]
g-rel_q088-1_t   [-2  0]
g-rel_q094-2_t   [-2  0]
g-rel_q097-2_t   [-2  0]
g-rel_q103-1_i   [-2  0]
g-rel_q116-1_r   [-2  0]
g-rel_q118-1_r   [-2  0]
g-rel_q122-2_i   [-2  0]
g-rel_q128-1_r   [-2  0]
g-rel_q134-3_t   [-2  0]


In [50]:
dataloader.grel_reading.keys()

dict_keys(['A04'])

In [51]:
article = dataloader.grel_articles[article_id]
gaze_data = dataloader.grel_reading['A04'][article_id[:-2]]['dataframe']
pars_mapping = mappingloader.paragraphs 
labels_mapping = mappingloader.labels

In [52]:
print(article.file_name, " ", any([par['answer'] for par in article.paragraphs]))

c:\Users\pablo\DFKI\scibot\scibot_visualization\server\src\data\articles\g-rel\main\g-rel_q088-1_t.html   False


In [40]:
for par_id in pars_mapping['paragraph_id'].to_numpy():
    print(par_id)

-2
0


In [65]:
article.article_id.lower()

'c:\\users\\pablo\\dfki\\scibot\\scibot_visualization\\server\\src\\data\\articles\\g-rel\\main\\g-rel_q088-1_t'

In [10]:
ids = list(range(-2, len(article.paragraphs)))
ids

[-2, -1, 0, 1, 2, 3, 4]

In [85]:
import os

from enum import Enum
from pandas.core.frame import DataFrame
from models.paragraph import ParagraphModel, ParagraphSchema
from marshmallow import Schema, fields
from typing import Any, List, Dict

from models import paragraph
from data_loading.article_parser import Article


class Corpus(Enum):
    grel = "g-REL"
    nq = "GoogleNQ"


class Document(object):
    """Document Representation"""

    def __init__(self, user_id: str, article_id: str, corpus: str, query: str,
                 paragraphs: List[ParagraphModel]):
        self.user_id = user_id
        self.id = article_id
        self.corpus = corpus
        self.query = query
        self.paragraphs = paragraphs

    @classmethod 
    def from_data(cls, user_id: str, article: Article, gaze_data: DataFrame, pars_mapping: DataFrame, labels_mapping: DataFrame):
        paragraphs = []
        _, article_id = os.path.split(article.article_id)

        # determine corpus
        corpus =  Corpus.grel if article_id.startswith('g-rel') else Corpus.nq

        # combine the HTML, gaze and mappping data to create the pararagrahs
        par_ids = pars_mapping['paragraph_id'].to_numpy()
        for par_id in par_ids: 
            if par_id >= 0: 
                if corpus == Corpus.grel: 
                    par_parsing = {'answer': any([par['answer'] for par in article.paragraphs])}
                else:
                    par_parsing = {'answer': article.paragraphs[par_id]['answer']}

            else:
                par_parsing = {'answer': False}

            par_gaze = gaze_data.loc[gaze_data['paragraph_id'] == par_id]
            par_mapping = pars_mapping.loc[pars_mapping['paragraph_id'] == par_id].to_numpy()[0]
            labels_selection = labels_mapping.loc[labels_mapping['paragraph_id'] == par_id]
            paragraphs.append(
                ParagraphModel.from_data(par_parsing, par_gaze, par_mapping, labels_selection)
            )
        
        return cls(user_id, article_id, corpus.value, article.query, paragraphs)

class DocumentSchema(Schema):
    user_id = fields.Str()
    id = fields.Str()
    corpus = fields.Str()
    query = fields.Str()
    paragraphs = fields.List(fields.Nested(ParagraphSchema))

In [86]:
document = Document.from_data('A01', article, gaze_data, pars_mapping, labels_mapping)

In [87]:
document.paragraphs[0]

In [88]:
document.corpus

'g-REL'